# yolov5自训练模型
## 自定义数据集

参照coco数据集的目录格式，创建数据集。

```bash
./my_dataset    # 数据集名称,可以自定义
├── data.py     # 数据集分集脚本
├── images      # 图像数据
├── labels      # 图像标注文件
├── train.txt   # 分集后用于训练的图像路径
└── val.txt     # 分集后用于验证的图像路径
```

## 数据准备
根据目标检测任务采集目标图像。为了提高数据集的有效性，需要从不同角度、不同位置、不同环境采集目标的图像。将其放入images目录中。

这里采集了300张四轮机器人oryx的图像，分为三个类别：oryx_logo、oryx_blue、oryx_red，各采集了100张图像。


## 目标检测任务数据标注

labelImage是一个用于目标检测任务的数据集标注软件
```bash
pip install labelImg
labelImg
```
点击**打开目录**按钮，选择路径：~/path/my_dataset/images。

点击**更改存放目录**按钮，选择路径：~/path/my_dataset/labels。

切换存储格式为**YOLO**。

![](images/2023-08-23-17-48-17.png)

点击**创建区块**按钮，框选目标区域，并且输入标签。

当标记完一张图片后，点击**保存**保存文件。

切换**下一张图片**，重复上诉操作

![](images/2023-08-23-17-49-47.png)

## 数据分集

在模型训练阶段，除了使用训练集训练模型，还需要验证集不断验证训练结果。

这里我们只使用训练集和验证集，分别需要创建train.txt和val.txt文件。

这里训练集和验证集的比例安装4:1的比例分集，为了使验证集和训练集的效果更好，分集前需要先将数据集打乱。

data.py文件：

```python
import random
import glob
import os


class1 = glob.glob("./images/logo*.jpg")
class2 = glob.glob("./images/oryx_blue*.jpg")
class3 = glob.glob("./images/red*.jpg")

# 打开分集文件
train = open('train.txt', 'w')
val = open('val.txt', 'w')

def spilt_data(data,rate):
    """
    数据分集，并写入train.txt和val.txt中

    param1 : 图像路径列表
    param2 : 分集比例
    """
    i = 0
    # 将列表打乱
    random.shuffle(data)  # 将图像列表打乱，提高数据的随机性
    for img in data:
        if i < len(data)*rate:
            train.write(img+'\n')  # 写入文件
        else:
            val.write(img+'\n')
        i += 1

spilt_data(class1,0.8)
spilt_data(class2,0.8)
spilt_data(class3,0.8)
```

现在my_dataset目录会生成train.txt和val.txt

## 修改数据集说明文件

进入yolov5/data/目录下，my_dataset.yaml文件。将数据集的信息按以下格式输入。

```yaml
path: ../datasets/my_dataset
train: train.txt
val: val.txt

names:
  0: oryx_logo
  1: oryx_blue
  2: oryx_red
```

## 修改模型组件文件

进入yolov5/models/目录下，复制yolov5s.yaml文件，并命名为yolov5s.yaml。

打开文件修改ns参数为检测的类别数量3。

## 模型训练
进入yolov5项目目录，终端运行下面指令。

In [1]:
import os
os.chdir("../yolov5")

train.py文件关键参数：
- weights：加载的权重文件
- data：数据集配置文件，数据集路径，类名等
- cfg：模型配置文件，网络结构
- epochs：训练总轮次
- batch-size：批次大小（根据设备性能选择，默认为16，可以相应减少）

In [2]:
!python train.py --weight yolov5s.pt --data data/my_dataset.yaml --cfg models/yolov5s_my.yaml --epochs 200 --batch-size 16

train: weights=yolov5s.pt, cfg=models/yolov5s_my.yaml, data=data/my_dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=200, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
Command 'git fetch ultralytics' timed out after 5 seconds
YOLOv5 🚀 v7.0-120-gea5702e Python-3.8.15 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5938MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0

## 模型测试

这里拍摄了一段测试视频，验证结果精度

detect.py文件关键参数：
- weights：权重文件路径
- source：测试数据，可以是图片/视频路径，也可以是'0'(电脑自带摄像头),也可以是rtsp等视频流
- conf：置信度阈值

In [3]:
!python detect.py --weight ./runs/train/exp/weights/best.pt --source ../datasets/my_dataset/test/oryx_all.avi

detect: weights=['./runs/train/exp/weights/best.pt'], source=../datasets/my_dataset/test/oryx_all.avi, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-120-gea5702e Python-3.8.15 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5938MiB)

Fusing layers... 
YOLOv5s_my summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/4936) /home/lyh/working_directory/visual_development/object_detection/datasets/my_dataset/test/oryx_all.avi: 480x640 1 oryx_blue, 1 oryx_red, 7.2ms
video 1/1 (2/4936) /home/lyh/working_directory/visual_development/object_detection/datasets/my_dataset/test/oryx_all.avi: 480x640 1 